# Graphics

In [ ]:
import numpy as np
from scipy import misc, ndimage
from skimage import feature, io, filters, morphology, measure
from skimage import data, color
import matplotlib.pyplot as plt
%matplotlib inline

## What is Computer Graphics?

## Image Basics
Images are made up of a grid of small colored squares called **Pixels**

In [ ]:
filename = "banana.jpg"
img = data.imread(filename,as_grey=True)
img_rgb = misc.imread(filename)

In [ ]:
plt.imshow(img_rgb)
plt.show()
print("shape of image RGB is " + str(img_rgb.shape))

### RGB Color Model

Images are made up of a 2 dimensional grid. The size of these dimensions are the first 2 numbers in the shape output above.

We call each cell in the grid a *pixel*. Each pixel has a location that is equal to its position in the x and y axis. The top left location is (0, 0)

If we zoom in far enough, we can see these pixels:

In [ ]:
plt.imshow(img_rgb, interpolation='none')
plt.xlim(0,5)
plt.ylim(0,5)
plt.show()

How does an image determine what color to display each pixel? This is what the third dimension of an image is for. Here is an example value of this dimension:

In [ ]:
print(img_rgb[0,0])

Each channel corresponds to the colors Red Green and Blue (where "RGB" comes from), and each value is how much of that color to mix.

In [ ]:
#print(str(img_rgb))

You can change the value of a pixel by setting it to a new combination of Red Green and Blue:

In [ ]:
width = 200
height = 200
new_drawing(width, height)
for x in range(width):
    for y in range(height):
        set_color(int(255 * (x/width)), int(255 * (y/height)), 128)
        set_pixel(x, y)
show()

## Graphics Techniques

Realism vs. Speed

### Raytracing

Pros, it can be used to create very realistic images

Cons, its very slow and cannot be used for realtime applications

### Polygons

Pros, it is very fast and most computers have specialized hardware to deal with polygon based graphics

Cons, it does not look as realistic

## Writing a Raytracer

[TODO: Write a tutorial for developing all of the code below]

In [ ]:
from drawing import *

# Useful math functions for dealing with vectors
# Not strictly necessary to understand these
def add(a, b):
    return (a[0] + b[0], a[1] + b[1], a[2] + b[2])

def scale(a, r):
    return (a[0] * r, a[1] * r, a[2] * r)

def dot(a, b):
    return a[0] * b[0] + a[1] * b[1] + a[2] * b[2]

def subtract(a, b):
    return add(a, scale(b, -1))

def normalize(a):
    return scale(a, 1/sqrt(dot(a,a)))

Our raytracer will only initially trace against spheres. Lets define a class in Python that allows us to keep track of these. The important attributes of a sphere will be its world coordinate location (position), its size (radius) and its color.

#### Sphere Class

For every type of object we want to render with the raytracer, we need to define how to intersect it with a ray. We do this by defining the method **intersect** on each shape type. We have provided an implementation of ray-sphere intersection in the Sphere class below. If you are curious about how it works, check out this wikipedia article (there are also many other resources online about the topic)

https://en.wikipedia.org/wiki/Line%E2%80%93sphere_intersection

We also need each type of object to tell the raytracer how to color it. We must define the function **colorize** for each shape type. For now, we will just return the color of the sphere and see what the results look like!

In [ ]:
class Sphere:
    def __init__(self, radius, position, color):
        self.radius = radius
        self.position = position
        self.color = color
    def intersect(self, source, direction, min_distance=0.01):
        v = subtract(source, self.position)
        b = -dot(v, direction)
        v2, r2 = dot(v,v), self.radius * self.radius
        d2 = b*b - v2 + r2
        if d2 > 0:
            for d in (b - sqrt(d2), b+sqrt(d2)):
                if d > min_distance:
                    return d
        return None
    def illumination(self, direction, surface, scene):
        return self.color

Now that we have a shape that we can raytrace, we need to be able to actually trace it! We will make this easier be defining a Scene class

#### Scene Class

The scene class allows us to keep track of all of the objects in the scene, as well as the camera and light positions. We will need attributes to keep track of all of these. Additionally, the Scene class will

In [ ]:
class Scene:
    def __init__(self, objects, lights, camera, ambient):
        self.objects = objects
        self.lights = lights
        self.camera = camera
        self.ambient = ambient
    def trace(self, source, direction):
        hits = []
        for s in self.objects:
            d = s.intersect(source,direction)
            if d is not None:
                hits.append((d, s))
        if not hits:
            return (0,0,0)
        distance, sphere = min(hits)
        surface = add(source, scale(direction, distance))
        return sphere.illumination(direction, surface, self)

def default_scene():
    objects = [
        Sphere(1, (0,0,5), (255,255,255)),
    ]
    lights = (2,2,0)
    camera = (0,0,1)
    ambient = 0.2
    return Scene(objects, lights, camera, ambient)
    
def render():
    width = 200
    height = 200
    scene = default_scene()
    new_drawing(width, height)
    for x in range(width):
        for y in range(height):
            direction = normalize((x/width-0.5, y/height-0.5, 1))
            color = scene.trace(scene.camera, direction)
            set_color(color[0], color[1], color[2])
            set_pixel(x, height-y)
    show()

In [ ]:
render()

We can modify the illumination function to change how the sphere gets its color. We will take into account the angle to the light that is in the scene object

[TODO: Insert graphic for this]

In [ ]:
def ill(self, direction, surface, scene):
    to_surface = subtract(surface, self.position)
    to_light = subtract(scene.lights, surface)
    intensity = max(scene.ambient,
                    dot(normalize(to_light), normalize(to_surface)))
    return scale(self.color, intensity)

Sphere.illumination = ill
render()

We can also modify the scene to include multiple objects. We do that by redefining the **default_scene** method. Here is an example of that.

In [ ]:
def default_scene():
    objects = [
        Sphere(1, (0,0,5), (0,255,255)),
        Sphere(1, (1,0,5), (255,0,255)),
    ]
    lights = (2,2,0)
    camera = (0,0,1)
    ambient = 0.2
    return Scene(objects, lights, camera, ambient)

render()

## What's next?
From here you can modify the ray tracer in countless ways. You could add new material types (reflective, different shading types, glass), you could add casted shadows, new shape types (planes, boxes), you can add textures.

Another option would be to programatically generate a scene. What are we able to render using just spheres? Could we general sphere locations/sizes using an algorithm to create a cool piece of art? Lets see an example of that...

In [ ]:
def default_scene():
    objects = []
    y = -5
    for i in range(1, 10):
        objects.append(Sphere(i/5, (0,y,15), (255-25*i,25*i,0)))
        y += i/5
    lights = (2,2,0)
    camera = (0,0,1)
    ambient = 0.2
    return Scene(objects, lights, camera, ambient)

render()

Some kind of worm?? Who knows!